# AutoML

PyCaret
* Main Site - https://pycaret.org/
* Docs - https://pycaret.readthedocs.io/en/latest/

## Table of Contents

* [Setup and Preprocessing](#setup)  
* [Compare Models](#compare)  
* [Create Model](#create)  
* [Tune Model](#tune)  
* [Evaluate Model](#evaluate)  
* [Finalize and Store Model](#finalize_and_store)  

## Imports and Global Settings

In [33]:
import mlflow
import pandas as pd
from sqlalchemy import create_engine
from pycaret.regression import *
from pycaret.classification import *

# Pandas Settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5


mlflow.set_tracking_uri('file:/home/jeff/Documents/Data_Science_Projects/NBA_Betting/models/AutoML')

## Database Connection

In [2]:
username = 'postgres'
password = ''
endpoint = ''
database = 'nba_betting'
port = '5432'

connection = create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

### Datasets

In [3]:
df = pd.read_sql_table('model_ready', connection)

<a id='basic_data_overview'></a>

## Basic Data Overview

In [4]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 1219 columns):
 #     Column                                 Non-Null Count  Dtype  
---    ------                                 --------------  -----  
 0     game_id                                9200 non-null   object 
 1     CLS_TARGET_home_margin_GT_home_spread  9200 non-null   bool   
 2     REG_TARGET_actual_home_margin          9200 non-null   float64
 3     home_team_num                          9200 non-null   int64  
 4     away_team_num                          9200 non-null   int64  
 5     league_year_end                        9200 non-null   int64  
 6     fd_line_home                           0 non-null      float64
 7     dk_line_home                           0 non-null      float64
 8     covers_consenses_home                  0 non-null      float64
 9     home_spread                            9200 non-null   float64
 10    gp                                     9172 

In [5]:
df.head()

,game_id,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,fd_line_home,dk_line_home,covers_consenses_home,home_spread,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_vla,gp_vla_std,win_rank,win_vla,win_vla_std,loss_rank,loss_vla,loss_vla_std,w_pct_rank,w_pct_vla,w_pct_vla_std,mins_rank,mins_vla,mins_vla_std,pts_rank,pts_vla,pts_vla_std,fgm_rank,fgm_vla,fgm_vla_std,fga_rank,fga_vla,fga_vla_std,fg_pct_rank,fg_pct_vla,fg_pct_vla_std,fg3m_rank,fg3m_vla,fg3m_vla_std,fg3a_rank,fg3a_vla,fg3a_vla_std,fg3_pct_rank,fg3_pct_vla,fg3_pct_vla_std,ftm_rank,ftm_vla,ftm_vla_std,fta_rank,fta_vla,fta_vla_std,ft_pct_rank,ft_pct_vla,ft_pct_vla_std,oreb_rank,oreb_vla,oreb_vla_std,dreb_rank,dreb_vla,dreb_vla_std,reb_rank,reb_vla,reb_vla_std,ast_rank,ast_vla,ast_vla_std,tov_rank,tov_vla,tov_vla_std,stl_rank,stl_vla,stl_vla_std,blk_rank,blk_vla,blk_vla_std,blka_rank,blka_vla,blka_vla_std,pf_rank,pf_vla,pf_vla_std,pfd_rank,pfd_vla,pfd_vla_std,p_m_rank,p_m_vla,p_m_vla_std,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_vla_opp,gp_vla_std_opp,win_rank_opp,win_vla_opp,win_vla_std_opp,loss_rank_opp,loss_vla_opp,loss_vla_std_opp,w_pct_rank_opp,w_pct_vla_opp,w_pct_vla_std_opp,mins_rank_opp,mins_vla_opp,mins_vla_std_opp,pts_rank_opp,pts_vla_opp,pts_vla_std_opp,fgm_rank_opp,fgm_vla_opp,fgm_vla_std_opp,fga_rank_opp,fga_vla_opp,fga_vla_std_opp,fg_pct_rank_opp,fg_pct_vla_opp,fg_pct_vla_std_opp,fg3m_rank_opp,fg3m_vla_opp,fg3m_vla_std_opp,fg3a_rank_opp,fg3a_vla_opp,fg3a_vla_std_opp,fg3_pct_rank_opp,fg3_pct_vla_opp,fg3_pct_vla_std_opp,ftm_rank_opp,ftm_vla_opp,ftm_vla_std_opp,fta_rank_opp,fta_vla_opp,fta_vla_std_opp,ft_pct_rank_opp,ft_pct_vla_opp,ft_pct_vla_std_opp,oreb_rank_opp,oreb_vla_opp,oreb_vla_std_opp,dreb_rank_opp,dreb_vla_opp,dreb_vla_std_opp,reb_rank_opp,reb_vla_opp,reb_vla_std_opp,ast_rank_opp,ast_vla_opp,ast_vla_std_opp,tov_rank_opp,tov_vla_opp,tov_vla_std_opp,stl_rank_opp,stl_vla_opp,stl_vla_std_opp,blk_rank_opp,blk_vla_opp,blk_vla_std_opp,blka_rank_opp,blka_vla_opp,blka_vla_std_opp,pf_rank_opp,pf_vla_opp,pf_vla_std_opp,pfd_rank_opp,pfd_vla_opp,pfd_vla_std_opp,p_m_rank_opp,p_m_vla_opp,p_m_vla_std_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,offrtg_rank,offrtg_vla,offrtg_vla_std,defrtg_rank,defrtg_vla,defrtg_vla_std,netrtg_rank,netrtg_vla,netrtg_vla_std,ast_pct_rank,ast_pct_vla,ast_pct_vla_std,ast_v_tov_rank,ast_v_tov_vla,ast_v_tov_vla_std,ast_ratio_rank,ast_ratio_vla,...,opp_fgm_c3,opp_fga_c3,opp_fg_pct_c3,opp_fgm_atb3,opp_fga_atb3,opp_fg_pct_atb3,opp_fgm_ra_rank,opp_fgm_ra_vla,opp_fgm_ra_vla_std,opp_fga_ra_rank,opp_fga_ra_vla,opp_fga_ra_vla_std,opp_fg_pct_ra_rank,opp_fg_pct_ra_vla,opp_fg_pct_ra_vla_std,opp_fgm_paint_rank,opp_fgm_paint_vla,opp_fgm_paint_vla_std,opp_fga_paint_rank,opp_fga_paint_vla,opp_fga_paint_vla_std,opp_fg_pct_paint_rank,opp_fg_pct_paint_vla,opp_fg_pct_paint_vla_std,opp_fgm_mr_rank,opp_fgm_mr_vla,opp_fgm_mr_vla_std,opp_fga_mr_rank,opp_fga_mr_vla,opp_fga_mr_vla_std,opp_fg_pct_mr_rank,opp_fg_pct_mr_vla,opp_fg_pct_mr_vla_std,opp_fgm_lc3_rank,opp_fgm_lc3_vla,opp_fgm_lc3_vla_std,opp_fga_lc3_rank,opp_fga_lc3_vla,opp_fga_lc3_vla_std,opp_fg_pct_lc3_rank,opp_fg_pct_lc3_vla,opp_fg_pct_lc3_vla_std,opp_fgm_rc3_rank,opp_fgm_rc3_vla,opp_fgm_rc3_vla_std,opp_fga_rc3_rank,opp_fga_rc3_vla,opp_fga_rc3_vla_std,opp_fg_pct_rc3_rank,opp_fg_pct_rc3_vla,opp_fg_pct_rc3_vla_std,opp_fgm_c3_rank,opp_fgm_c3_vla,opp_fgm_c3_vla_std,opp_fga_c3_rank,opp_fga_c3_vla,opp_fga_c3_vla_std,opp_fg_pct_c3_rank,opp_fg_pct_c3_vla,opp_fg_pct_c3_vla_std,opp_fgm_atb3_rank,opp_fgm_atb3_vla,opp_fgm_atb3_vla_std,opp_fga_atb3_rank,opp_fga_atb3_vla,opp_fga_atb3_vla_std,opp_fg_pct_atb3

## PyCaret - Regression

<a id=setup></a>

### Setup and Preprocessing

In [16]:
features_to_drop = ['game_id', 'CLS_TARGET_home_margin_GT_home_spread']
keep_features = ['REG_TARGET_actual_home_margin',
                 'home_team_num', 'away_team_num',
                 'league_year_end', 'home_spread']
features_to_use = [feature for feature in list(df) if feature[-3:] == 'vla'] + keep_features

In [17]:
model_ready_df = df.drop(columns=features_to_drop)
model_ready_df = model_ready_df[features_to_use]

In [18]:
model_ready_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 156 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    gp_vla                         9172 non-null   float64
 1    win_vla                        9172 non-null   float64
 2    loss_vla                       9172 non-null   float64
 3    w_pct_vla                      9172 non-null   float64
 4    mins_vla                       9172 non-null   float64
 5    pts_vla                        9172 non-null   float64
 6    fgm_vla                        9172 non-null   float64
 7    fga_vla                        9172 non-null   float64
 8    fg_pct_vla                     9172 non-null   float64
 9    fg3m_vla                       9172 non-null   float64
 10   fg3a_vla                       9172 non-null   float64
 11   fg3_pct_vla                    9172 non-null   float64
 12   ftm_vla                        9

The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [21]:
setup_params = {'log_experiment': True,
                'log_profile': False,
                'log_plots': False,
                'experiment_name': 'NBA_Betting_REG_vla_only',
                'data': model_ready_df,
                'target': 'REG_TARGET_actual_home_margin',
                'train_size': 0.7,
                'preprocess': True,
                'normalize': False,        # zscore
                'transformation': False,   # yeo-johnson power transform to make data more Gaussian
                'remove_outliers': False,  # using SVD
                'remove_multicollinearity': False,
                'polynomial_features': False,
                'trigonometry_features': False,
                'feature_interaction': False,
                'feature_ratio': False,
                'feature_selection': False,
                'feature_selection_threshold': 0.8,
                'pca': False,
                'pca_components': 10,
                'numeric_features': ['league_year_end'],
                'ignore_features': []
               }

In [22]:
nba_betting_regression = setup(**setup_params)

,Description,Value
0,session_id,6552
1,Target,REG_TARGET_actual_home_margin
2,Original Data,"(9200, 156)"
3,Missing Values,True
4,Numeric Features,155
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(6439, 149)"


Traceback (most recent call last):
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 336, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 175, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/jeff/Documents/Data_Science_Projects/NBA_Betting/models/AutoML/mlruns/meta.yaml' does not exist.
Traceback (most recent call last):
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/jeff/anaconda3/l

INFO:logs:setup() succesfully completed......................................


<a id=compare></a>

### Compare Models

In [23]:
best_3_models = compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,9.7694,156.9964,12.5235,0.2171,0.9843,1.1380,0.0250
en,Elastic Net,9.7690,157.0419,12.5253,0.2169,0.9942,1.1385,0.0260
br,Bayesian Ridge,9.7892,157.7518,12.5539,0.2132,1.0188,1.1422,0.1310
omp,Orthogonal Matching Pursuit,9.7985,158.0832,12.5669,0.2116,1.0140,1.1542,0.0250
huber,Huber Regressor,9.8449,158.8296,12.5972,0.2076,1.0276,1.1639,0.6220
ridge,Ridge Regression,9.8747,159.9541,12.6413,0.2021,1.0362,1.1756,0.0270
gbr,Gradient Boosting Regressor,9.8990,160.4637,12.6624,0.1994,1.0690,1.1491,9.6210
ada,AdaBoost Regressor,9.9950,162.4208,12.7384,0.1898,1.1407,1.1282,3.2730
lightgbm,Light Gradient Boosting Machine,10.2043,169.0197,12.9953,0.1566,1.0764,1.2361,1.0850
catboost,CatBoost Regressor,10.2243,170.0664,13.0358,0.1510,1.0974,1.2467,24.5650


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 2
INFO:logs:[Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=6552,
      selection='cyclic', tol=0.0001, warm_start=False), ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=6552, selection='cyclic', tol=0.0001, warm_start=False), BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)]
INFO:logs:compare_models() succesfully completed......................................


<a id=create></a>

### Create Selected Model

In [24]:
model = create_model('lasso')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,9.8873,157.6939,12.5576,0.2433,0.9981,1.1097
1,9.5249,151.0432,12.2900,0.2326,1.0174,1.1906
2,9.8106,169.6824,13.0262,0.2241,0.9973,1.0808
3,10.3259,167.4848,12.9416,0.1905,0.9913,1.1880
4,9.4547,138.1313,11.7529,0.2164,0.9660,1.2362
5,9.5364,150.5830,12.2712,0.1788,0.9574,1.1203
6,9.5775,155.4738,12.4689,0.2481,0.9599,1.0388
7,9.7286,158.3967,12.5856,0.2217,0.9728,1.0442
8,10.2849,171.7997,13.1072,0.2062,1.0313,1.1783
9,9.5636,149.6755,12.2342,0.2094,0.9510,1.1929


INFO:logs:create_model_container: 21
INFO:logs:master_model_container: 21
INFO:logs:display_container: 3
INFO:logs:Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=6552,
      selection='cyclic', tol=0.0001, warm_start=False)
INFO:logs:create_model() succesfully completed......................................


<a id=tune></a>

### Tune Selected Model

In [25]:
tuned_model = tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,9.8727,158.0428,12.5715,0.2416,1.0002,1.0893
1,9.5475,151.4192,12.3053,0.2307,1.0181,1.1816
2,9.8299,170.1477,13.0441,0.2219,0.9977,1.0676
3,10.2944,166.8281,12.9162,0.1937,0.9888,1.1675
4,9.4806,138.3340,11.7615,0.2153,0.9653,1.2278
5,9.5168,150.2469,12.2575,0.1806,0.9556,1.1020
6,9.6031,156.3401,12.5036,0.2439,0.9649,1.0304
7,9.7504,159.0207,12.6103,0.2187,0.9767,1.0399
8,10.2997,172.1793,13.1217,0.2044,1.0310,1.1634
9,9.5835,149.9318,12.2447,0.2080,0.9573,1.1855


INFO:logs:create_model_container: 22
INFO:logs:master_model_container: 22
INFO:logs:display_container: 4
INFO:logs:Lasso(alpha=2.25, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=6552,
      selection='cyclic', tol=0.0001, warm_start=False)
INFO:logs:tune_model() succesfully completed......................................


<a id=evaluate></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [26]:
evaluate_model(tuned_model)

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=Lasso(alpha=2.25, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=6552,
      selection='cyclic', tol=0.0001, warm_start=False), fold=None, fit_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [27]:
# interpret_model(tuned_model)

<a id=finalize_and_store></a>

### Model Finalization and Storage

In [28]:
final_model = finalize_model(tuned_model)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=Lasso(alpha=2.25, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=6552,
      selection='cyclic', tol=0.0001, warm_start=False), fit_kwargs=None, groups=None, model_only=True, display=None)
INFO:logs:Finalizing Lasso(alpha=2.25, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=6552,
      selection='cyclic', tol=0.0001, warm_start=False)
INFO:logs:Initializing create_model()
INFO:logs:create_model(estimator=Lasso(alpha=2.25, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=6552,
      selection='cyclic', tol=0.0001, warm_start=False), fold=None, round=4, cross_validation=True, predict=True, fit_kwargs={}, groups=None, refit=True, verbose=False, system=False, metrics=None, add_to_model_list=False, probability

In [29]:
save_model(final_model, '../models/AutoML/Baseline_Lasso_Reg_PyCaret')

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=Lasso(alpha=2.25, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=6552,
      selection='cyclic', tol=0.0001, warm_start=False), model_name=../models/AutoML/Baseline_Lasso_Reg_PyCaret, prep_pipe_=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['league_year_end'],
                                      target='REG_TARGET_actual_home_margin',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                              

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['league_year_end'],
                                       target='REG_TARGET_actual_home_margin',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fil...
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  Lasso(alpha=2.25, copy_X=True, fit_intercept=True,
  

In [ ]:
# !mlflow ui

## Classification

<a id=setup></a>

### Setup and Preprocessing

In [30]:
features_to_drop = ['game_id', 'REG_TARGET_actual_home_margin']
keep_features = ['CLS_TARGET_home_margin_GT_home_spread',
                 'home_team_num', 'away_team_num',
                 'league_year_end', 'home_spread']
features_to_use = [feature for feature in list(df) if feature[-3:] == 'vla'] + keep_features

In [31]:
model_ready_df = df.drop(columns=features_to_drop)
model_ready_df = model_ready_df[features_to_use]

In [32]:
model_ready_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 156 columns):
 #    Column                                 Non-Null Count  Dtype  
---   ------                                 --------------  -----  
 0    gp_vla                                 9172 non-null   float64
 1    win_vla                                9172 non-null   float64
 2    loss_vla                               9172 non-null   float64
 3    w_pct_vla                              9172 non-null   float64
 4    mins_vla                               9172 non-null   float64
 5    pts_vla                                9172 non-null   float64
 6    fgm_vla                                9172 non-null   float64
 7    fga_vla                                9172 non-null   float64
 8    fg_pct_vla                             9172 non-null   float64
 9    fg3m_vla                               9172 non-null   float64
 10   fg3a_vla                               9172 non-null   flo

The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [34]:
setup_params = {'log_experiment': True,
                'log_profile': False,
                'log_plots': False,
                'experiment_name': 'NBA_Betting_CLS_vla_only',
                'data': model_ready_df,
                'target': 'CLS_TARGET_home_margin_GT_home_spread',
                'train_size': 0.7,
                'preprocess': True,
                'normalize': False,        # zscore
                'transformation': False,   # yeo-johnson power transform to make data more Gaussian
                'remove_outliers': False,  # using SVD
                'remove_multicollinearity': False,
                'polynomial_features': False,
                'trigonometry_features': False,
                'feature_interaction': False,
                'feature_ratio': False,
                'feature_selection': False,
                'feature_selection_threshold': 0.8,
                'pca': False,
                'pca_components': 10,
                'numeric_features': ['league_year_end'],
                'ignore_features': []
               }

In [35]:
nba_betting_classification = setup(**setup_params)

,Description,Value
0,session_id,6420
1,Target,CLS_TARGET_home_margin_GT_home_spread
2,Target Type,Binary
3,Label Encoded,"False: 0, True: 1"
4,Original Data,"(9200, 156)"
5,Missing Values,True
6,Numeric Features,155
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:Logging experiment in MLFlow
Traceback (most recent call last):
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 336, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 175, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/jeff/Documents/Data_Science_Projects/NBA_Betting/models/AutoML/mlruns/meta.yaml' does not exist.
Traceback (most recent call last):
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, vi

INFO:logs:setup() succesfully completed......................................


<a id=compare></a>

### Compare Models

In [36]:
best_3_models = compare_models(n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.5198,0.5206,0.4387,0.5081,0.4704,0.0355,0.0360,0.1580
lr,Logistic Regression,0.5155,0.5217,0.4378,0.5024,0.4676,0.0270,0.0272,1.1800
ridge,Ridge Classifier,0.5151,0.0000,0.4390,0.5019,0.4681,0.0264,0.0266,0.0290
dummy,Dummy Classifier,0.5133,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0240
nb,Naive Bayes,0.5113,0.5112,0.3258,0.4970,0.3913,0.0130,0.0139,0.0270
ada,Ada Boost Classifier,0.5052,0.4974,0.4343,0.4908,0.4603,0.0068,0.0069,1.7640
lightgbm,Light Gradient Boosting Machine,0.5046,0.5005,0.4607,0.4904,0.4749,0.0069,0.0069,0.9900
dt,Decision Tree Classifier,0.5021,0.5019,0.4958,0.4884,0.4917,0.0038,0.0038,0.4630
xgboost,Extreme Gradient Boosting,0.5016,0.4992,0.4777,0.4876,0.4824,0.0020,0.0020,51.5210
et,Extra Trees Classifier,0.5013,0.5013,0.4722,0.4874,0.4795,0.0011,0.0011,1.0890


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 2
INFO:logs:[LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001), LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6420, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=6420,
                solver='auto', tol=0.001)]
INFO:logs:compare_models() succesfully completed......................................


<a id=create></a>

### Create Selected Model

In [37]:
model = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5078,0.5053,0.4409,0.4929,0.4654,0.0119,0.0120
1,0.5016,0.5332,0.3994,0.4845,0.4378,-0.0025,-0.0025
2,0.5062,0.5202,0.3834,0.4898,0.4301,0.0058,0.0059
3,0.5295,0.5328,0.4824,0.5171,0.4992,0.0565,0.0567
4,0.5109,0.5058,0.4377,0.4964,0.4652,0.0178,0.0179
5,0.5171,0.5343,0.4299,0.5056,0.4647,0.0300,0.0304
6,0.5155,0.5048,0.4395,0.5036,0.4694,0.0275,0.0277
7,0.5497,0.5529,0.4745,0.5438,0.5068,0.0960,0.0968
8,0.4922,0.4933,0.4427,0.4777,0.4595,-0.0180,-0.0180
9,0.5241,0.5346,0.4473,0.5128,0.4778,0.0444,0.0448


INFO:logs:create_model_container: 17
INFO:logs:master_model_container: 17
INFO:logs:display_container: 3
INFO:logs:LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6420, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:create_model() succesfully completed......................................


<a id=tune></a>

### Tune Selected Model

In [38]:
tuned_model = tune_model(model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5031,0.5037,0.4441,0.4877,0.4649,0.0030,0.0030
1,0.5031,0.5312,0.3994,0.4864,0.4386,0.0006,0.0006
2,0.5062,0.5222,0.3866,0.4899,0.4321,0.0060,0.0061
3,0.5373,0.5324,0.4920,0.5256,0.5083,0.0722,0.0723
4,0.5140,0.5060,0.4473,0.5000,0.4722,0.0244,0.0245
5,0.5140,0.5327,0.4236,0.5019,0.4594,0.0237,0.0239
6,0.5155,0.5037,0.4395,0.5036,0.4694,0.0275,0.0277
7,0.5466,0.5519,0.4682,0.5404,0.5017,0.0897,0.0904
8,0.4984,0.4917,0.4427,0.4843,0.4626,-0.0058,-0.0058
9,0.5303,0.5339,0.4569,0.5200,0.4864,0.0570,0.0575


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 4
INFO:logs:LogisticRegression(C=4.394, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6420, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:tune_model() succesfully completed......................................


<a id=evaluate></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [39]:
evaluate_model(tuned_model)

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=LogisticRegression(C=4.394, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6420, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), fold=None, fit_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [40]:
# interpret_model(tuned_model)

<a id=finalize_and_store></a>

### Model Finalization and Storage

In [41]:
final_model = finalize_model(tuned_model)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=LogisticRegression(C=4.394, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6420, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), fit_kwargs=None, groups=None, model_only=True, display=None)
INFO:logs:Finalizing LogisticRegression(C=4.394, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6420, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:Initializing create_model()
INFO:logs:create_model(estimator=LogisticRegression(C=4.394, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, 

In [42]:
save_model(final_model, '../models/AutoML/Baseline_LR_CLS_PyCaret')

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=LogisticRegression(C=4.394, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6420, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), model_name=../models/AutoML/Baseline_LR_CLS_PyCaret, prep_pipe_=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['league_year_end'],
                                      target='CLS_TARGET_home_margin_GT_home_spread',
                                      time_features=[])),
             

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['league_year_end'],
                                       target='CLS_TARGET_home_margin_GT_home_spread',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categori...
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LogisticRegression(C=4.394, class_weight={}, dual=False,
                                     fit_inter

In [ ]:
# !mlflow ui